# [LVV-T2213] - Look-up Table Application from MTMount Elevation Changes

This notebook was originally written by Bo Xin in the [lsst-ts/ts_notebooks] repository.  
It is a modified version with updated commands and simplified steps.

**Make sure you run this notebook on TTS before running at the summit.**

[lsst-ts/ts_notebooks]: https://github.com/lsst-ts/ts_notebooks/blob/develop/bxin/aos2comp/aos2comp.ipynb
[LVV-T2213]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2213

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lsst.sitcom.vandv import info

exec_info = info.ExecutionInfo()
print(exec_info)


Executed by b1quint on 2022-05-27T16:07:52.640.
  Running in pillan04 at tucson



---
## Setup Notebook for Test

- Import all libraries
- Get the remotes ready

In [3]:
import asyncio
import os
import yaml

import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
from astropy.time import Time
from datetime import datetime, timedelta

from lsst_efd_client import EfdClient

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel.mtcs import MTCS, MTCSUsages
from lsst.ts.observatory.control import RotType
from lsst.sitcom.vandv import aos_tools

In [4]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])
print(os.environ["LSST_DDS_DOMAIN_ID"])

file:///opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-3.0.0/lib/python3.8/config/ospl-shmem.xml
tucson
0


In [5]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [6]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

In [7]:
domain = salobj.Domain()

In [8]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(40)

In [9]:
await mtcs.start_task

[None, None, None, None, None, None, None, None, None, None]

## Start all the components, and put them in an enabled state.

Depending on which test cycle this is being executed in, each component is either a hardware component or a simulator:

- [ ] M1M3
- [ ] M2 
- [ ] M2 Hexapod
- [ ] Camera Hexapod
- [ ] MTMount

The best/simplest way of doing this is running the notebook that executes the [LVV-T2344] test case.  
This notebook puts all the components in a ENABLED state.

[LVV-T2344]: https://github.com/lsst-sitcom/notebooks_vandv/blob/develop/notebooks/proj_sys_eng/sitcom_integration/l3_system_integ/LVV-T2344-Startup_mt_on_level3.ipynb

In [10]:
# Verify that all the components have heartbeats
await mtcs.assert_liveliness()

In [11]:
# Verify that all the components are enabled
await mtcs.assert_all_enabled()

---
## Move mtmount to Zenith

Command the mount to elevation = 90, azimuth = 0, so that we can start m1m3 with LUT in mount telemetry mode).

In [ ]:
await mtcs.rem.mtmount.cmd_moveToTarget.set_start(azimuth=0, elevation=90)

---
## Get M1M3 Ready

- Raise the mirror,
- Turn on Balance Forces,
- Clear forces

Need to have M1M3 LUT use mount telemetry.

In [ ]:
# If M1M3 is not raised yet, use this command to raise it.
await mtcs.raise_m1m3()

In [ ]:
# Enables M1M3 Force Balance system using the hardpoints
await mtcs.enable_m1m3_balance_system()

In [ ]:
# Resets the Aberration Forces and the Active Optics Forces
await mtcs.reset_m1m3_forces()

In [ ]:
# M1M3 LUT use mount telemetry
# todo: how to do that? 

## Get M2 Ready
- Turn on Force Balance system
- Clear forces

In [ ]:
# Enabled M2 Force Balance system 
await mtcs.enable_m2_balance_system()

In [ ]:
# Resets the Active Optics Forces
await mtcs.reset_m2_forces()

In [ ]:
# Need to have M2 LUT use mount telemetry
# todo: how to do that?

## Get CamHex Ready
- Check config 
- Make sure LUT is on, and has valid inputs
- Make sure hex is at LUT position

In [35]:
# Check the configuration
print(mtcs.rem.mthexapod_1.evt_configurationApplied.get())

camhex_config = await mtcs.rem.mthexapod_1.evt_configuration.aget(timeout=10.)
print(
    f"\nPivot at ({hexaConfig.pivotX}, {hexaConfig.pivotY}, {hexaConfig.pivotZ}) microns "
    f"\n maxXY = {hexaConfig.maxXY} microns, maxZ = {hexaConfig.maxZ} microns"
    f"\n maxUV = {hexaConfig.maxUV} deg, maxW = {hexaConfig.maxW} deg"
)

MTHexapodID: 1, private_revCode: 047bbc0a, private_sndStamp: 1652662975.6758816, private_rcvStamp: 1653667674.6886544, private_seqNum: 1, private_identity: MTHexapod:1, private_origin: 22691, configurations: _init.yaml, version: v0.8.1-0-g0400d07, url: file:///home/saluser/ts_config_mttcs/MTHexapod/v3, schemaVersion: v3, otherInfo: , priority: 0

Pivot at (0.0, 0.0, 500000.0) microns 
 maxXY = 11400.0 microns, maxZ = 13100.0 microns
 maxUV = 0.36 deg, maxW = 0.1 deg


In [ ]:
# Enable compensation mode for CamHex
await mtcs.enable_compensation_mode("mthexapod_1")

In [ ]:
# Reset the Camera Hexapod position
await mtcs.reset_camera_hexapod_position()

In [36]:
# Print Compensation Position and Uncompensation Position
posU = await mtcs.rem.mthexapod_1.evt_uncompensatedPosition.aget(timeout=10.)
print('Uncompensated position')
print(" ".join(f"{p:10.2f}" for p in [getattr(posU, i) for i in 'xyz']), end = '    ')
print(" ".join(f"{p:10.6f}" for p in [getattr(posU, i) for i in 'uvw']),'  ',
     pd.to_datetime(posU.private_sndStamp, unit='s'))    

posC = await mtcs.rem.mthexapod_1.evt_compensatedPosition.aget(timeout=10.)
print('Compensated position')
print(" ".join(f"{p:10.2f}" for p in [getattr(posC, i) for i in 'xyz']), end = '     ')
print(" ".join(f"{p:10.6f}" for p in [getattr(posC, i) for i in 'uvw']),'  ',
     pd.to_datetime(posC.private_sndStamp, unit='s'))

Uncompensated position
     -0.00      -0.00      29.07     -0.000000  -0.000000   0.000000    2022-05-27 08:27:05.493720576
Compensated position
     -0.00      -0.00      29.07      -0.000000  -0.000000   0.000000    2022-05-27 08:27:05.493976832


In [45]:
print("Does the hexapod has enough inputs to do LUT compensation? (If the below times out, we do not.)")
# Note: the target events are what the hexa CSC checks; if one is missing, the entire LUT will not be applied
# It also needs to see an uncompensatedPosition (a move would trigger that) in order to move to the compensatedPosition
a = await mtcs.rem.mthexapod_1.evt_compensationOffset.aget(timeout=10.)

print("mount elevation = ", a.elevation)
print("mount azimth = ", a.azimuth)
print("rotator angle = ", a.rotation)
print("? temperature = ", a.temperature)
print("\n".join(f"{i} = {getattr(a, i):.2f}" for i in "xyzuvw"))

Does the hexapod has enough inputs to do LUT compensation? (If the below times out, we do not.)
mount elevation =  80.0
mount azimth =  0.0
rotator angle =  0.0
? temperature =  0.0
x = -0.87
y = -423.42
z = 283.37
u = -0.01
v = 0.00
w = 0.00


## Get M2Hex Ready

- Check config 
- Make sure LUT is on, and has valid inputs
- Make sure M2Hex is at LUT position

In [ ]:
# Check the configuration
print(mtcs.rem.mthexapod_2.evt_configurationApplied.get())

camhex_config = await mtcs.rem.mthexapod_1.evt_configuration.aget(timeout=10.)
print(
    f"\nPivot at ({hexaConfig.pivotX}, {hexaConfig.pivotY}, {hexaConfig.pivotZ}) microns "
    f"\n maxXY = {hexaConfig.maxXY} microns, maxZ = {hexaConfig.maxZ} microns"
    f"\n maxUV = {hexaConfig.maxUV} deg, maxW = {hexaConfig.maxW} deg"
)

In [ ]:
# Enable compensation mode for M2Hex
await mtcs.enable_compensation_mode("mthexapod_2")

In [ ]:
# Reset the M2 Hexapod position
await mtcs.reset_m2_hexapod_position()

In [ ]:
# Print Compensation Position and Uncompensation Position
posU = await mtcs.rem.mthexapod_2.evt_uncompensatedPosition.aget(timeout=10.)
print('Uncompensated position')
print(" ".join(f"{p:10.2f}" for p in [getattr(posU, i) for i in 'xyz']), end = '    ')
print(" ".join(f"{p:10.6f}" for p in [getattr(posU, i) for i in 'uvw']),'  ',
     pd.to_datetime(posU.private_sndStamp, unit='s'))    

posC = await mtcs.rem.mthexapod_2.evt_compensatedPosition.aget(timeout=10.)
print('Compensated position')
print(" ".join(f"{p:10.2f}" for p in [getattr(posC, i) for i in 'xyz']), end = '     ')
print(" ".join(f"{p:10.6f}" for p in [getattr(posC, i) for i in 'uvw']),'  ',
     pd.to_datetime(posC.private_sndStamp, unit='s'))

In [ ]:
print("Does the hexapod has enough inputs to do LUT compensation? (If the below times out, we do not.)")
# Note: the target events are what the hexa CSC checks; if one is missing, the entire LUT will not be applied
# It also needs to see an uncompensatedPosition (a move would trigger that) in order to move to the compensatedPosition
a = await mtcs.rem.mthexapod_2.evt_compensationOffset.aget(timeout=10.)

print("mount elevation = ", a.elevation)
print("mount azimth = ", a.azimuth)
print("rotator angle = ", a.rotation)
print("? temperature = ", a.temperature)
print("\n".join(f"{i} = {getattr(a, i):.2f}" for i in "xyzuvw"))

## Gather Data - No AO

  * command the mount to elevation =86 deg, azimuth = 0
  * wait 39s
  * command the mount to elevation = 82 deg, azimuth = 0.

### Step 1: add aberrations via OFC

In [ ]:
zernikes = np.zeros(19)
zerIdx = np.arange(4, 23)
zernikes[7-4] = 1 #add 1um of z7
print(zerIdx)
print(zernikes)

In [ ]:
#x0 makes sure corrections double when same aberrations are added twice
config = {"xref": "x0"}  
config_yaml = yaml.safe_dump(config)

In [ ]:
await aos.cmd_addAberration.set_start(wf = zernikes, config=config_yaml)

In [ ]:
ofc_dict =  await ofcSentApplied(aos, m1m3, m2, camhex, m2hex, make_plot = True)

In [ ]:
await aos.cmd_issueCorrection.set_start() 

In [ ]:
ofc_dict =  await ofcSentApplied(aos, m1m3, m2, camhex, m2hex, make_plot = True)

#### Look at telemetry to verify the changes in the DOFs

In [ ]:
end = Time(datetime.now(), scale='tai')
#end = Time('2021-05-14T18:25:30', scale = 'tai')
start = end - timedelta(seconds=100)

dfm = await client.select_time_series('lsst.sal.MTMount.elevation', '*', start, end, csc_index)
dfm1m3 = await client.select_time_series('lsst.sal.MTM1M3.logevent_appliedActiveOpticForces', '*', start, end, csc_index)
dfm2 = await client.select_time_series('lsst.sal.MTM2.axialForce', '*', start, end, csc_index)
dfh = await client.select_time_series('lsst.sal.MTHexapod.logevent_uncompensatedPosition', '*', start, end, csc_index)

idx1=dfh.MTHexapodID==1
dfh1 = dfh[idx1]
idx2=dfh.MTHexapodID==2
dfh2 = dfh[idx2]

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,4))
#plt.plot(dfm.actualPosition, '--', label='mount elevation')
plt.plot(dfm1m3.zForces0*1e3, label='M1M3 aos force 101 (x1000N)')
plt.plot(dfm2.applied0*1e2, label='M2 aos force B1 (x100N)')
plt.plot(dfh1.y, '*', label='Camera hexapod y (microns)')
plt.plot(dfh2.y, 'o', label='M2 hexapod y (microns)')
plt.grid()
plt.legend()

### Step 2: Reset the corrections

In [ ]:
await aos.cmd_resetCorrection.set_start()
await aos.cmd_issueCorrection.set_start() 

In [ ]:
ofc_dict3 =  await ofcSentApplied(aos, m1m3, m2, camhex, m2hex, make_plot = True)

### Step 3: Double the aberration, do the corrections double? yes

In [ ]:
zernikes[7-4] = 2 # z7 = 2um

In [ ]:
await aos.cmd_addAberration.set_start(wf = zernikes) #config=??

In [ ]:
await aos.cmd_issueCorrection.set_start() 

In [ ]:
ofc_dict4 =  await ofcSentApplied(aos, m1m3, m2, camhex, m2hex, make_plot = True)

In [ ]:
ofc_dict4['camhexC'][:5]/ofc_dict['camhexC'][:5]

In [ ]:
ofc_dict4['m2hexC'][:5]/ofc_dict['m2hexC'][:5]

### Close up

In [ ]:
#if we started with enabled state, we need to put it back
await salobj.set_summary_state(m1m3, salobj.State.ENABLED,  settingsToApply='Default')

In [ ]:
await salobj.set_summary_state(m2, salobj.State.OFFLINE)

In [ ]:
await salobj.set_summary_state(m1m3, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(camhex, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(aos, salobj.State.STANDBY)

In [ ]:
await lowerM1M3(m1m3)

### Check if the telescope is in tracking mode. If yes, need to stop stacking. 
The alternative is to check "MT Mount status" dash board on Chronograf. Make sure there are three "False".

In [ ]:
mountStatus = await mount.evt_axesInPosition.aget(timeout=5.)
rotStatus = await rot.evt_inPosition.aget(timeout=5.)
trackingStatus = mountStatus.elevation and mountStatus.azimuth and rotStatus.inPosition
print('Are we tracking?', trackingStatus)

In [ ]:
await ptg.cmd_stopTracking.set_start(timeout=5.)